# Data preparation

In [1]:
import pandas as pd
import numpy as np

import os

import warnings

from typing import List, Dict

## Auxilliary functions
The following section defines some functions which will be later used in processing the dataset.

In [2]:
def read_forest_data(file_name:str) -> np.ndarray:
    '''Reads the data on plant browsing from an Excel sheet into a numpy array.
    
    :param: filename: a string containing the name of the Excel file.
    :return: a numpy array containing the data.
    '''

    # Constants used in the function:
    START_COLUMN = 'P'                          # The column in which the data starts in the Excel sheet.
    NAME_INDEX = 1                              # Index of the column with the first species name.
    SAMPLING_POINTS = 100                       # Number of sampling points.
    SHEET_NAME = 'terepi-hajtásszám&hullaték'

    df = pd.read_excel(file_name, sheet_name=SHEET_NAME, skiprows=3, usecols=f'{START_COLUMN}:ZZZ', header=None, engine='openpyxl').reset_index(drop=True)

    last_column = df.iloc[1].tolist().index('Kínált hajtások száma az egyes pontokban') - 1

    # Initialize the numpy array
    num_rows = SAMPLING_POINTS + 3              # 100 sampling points + 3 headers
    num_cols = (last_column + 1)
    data = np.zeros((num_rows, num_cols), dtype=object)

    # Populate the array with values from the dataframe.
    for i in range(len(data)):
        
        # Species names are inserted into the first row.
        if i == 0:
            name_index = NAME_INDEX

            while name_index < last_column:
                data[i, name_index-1:name_index+8] = df.iloc[0,name_index]

                name_index += 9

        # Count headers are inserted into the second row.
        elif i == 1:
            repeats = num_cols / 9

            iteration = 0

            while iteration < repeats:
                start_point = iteration * 9

                data[i, start_point:start_point+4] = 'Összes hajtásvég'
                data[i, start_point+4:start_point+8] = 'Friss rágott hajtásvég'
                data[i, start_point+8] = 'Régi rágott hajtásvég'

                iteration += 1

        
        # The rest of the dataframe is copied into the array.
        else:
            data[i, :] = df.iloc[i, :last_column + 1].values

    return data

In [21]:
# Testing read_forest_data.
test_data = read_forest_data('Forests/Nyugat_Mátra/Ny_Mátra 2 vonal.xlsx')
np.savetxt('./Tests/test_read_forest_data.csv', test_data, fmt='%s', delimiter=',')

/var/folders/t3/fn5x2gps4wq728lxf9qrv_lm0000gn/T/ipykernel_40721/2886446634.py:14: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  df = pd.read_excel(file_name, sheet_name=SHEET_NAME, skiprows=3, usecols=f'{START_COLUMN}:ZZZ', header=None, engine='openpyxl').reset_index(drop=True)


In [3]:
log = []

def aggregate_data(data:np.ndarray) -> np.ndarray:
    '''Aggregates height levels in forest browsing data.
    
    :param: data: numpy array containing unaggregated data.
    :return: a numpy array containing the aggregated data.
    '''

    # Initialize new array.
    nr_rows = np.shape(data)[0] - 2             # Two less rows as original data since no height level and measurement headers are removed.
    
    nr_species = int(np.shape(data)[1] / 9)
    nr_columns = nr_species * 3

    aggregated = np.zeros((nr_rows, nr_columns), dtype=object)

    # Get species names.
    unique_species, indices = np.unique(data[0], return_index=True)
    species = unique_species[np.argsort(indices)].tolist()

    for i in range(nr_rows):
        if i == 0:
            for index, sp in enumerate(species):
                start = index * 3
                aggregated[i, start] = sp + ' all shoots'
                aggregated[i, start+1] = sp + ' freshly browsed'
                aggregated[i, start+2] = sp + ' old browsed'

        else:
            for j in range(nr_species):
                start_unaggregated = j * 9
                start_aggregated = j * 3

                aggregated_shoots = np.sum(data[i + 2, start_unaggregated:start_unaggregated+4])
                aggregated_browsed = np.sum(data[i + 2, start_unaggregated+4:start_unaggregated+8])

                if aggregated_browsed > aggregated_shoots:
                    log.append(f'{species[j]}\t ROW {i}\t ALL: {aggregated_shoots}\t FRESH: {aggregated_browsed}')
                    
                    # If more browsed than offered is recorded, the number offered is corrected to offered + browsed.
                    aggregated[i, start_aggregated] = aggregated_shoots + aggregated_browsed

                else:
                    aggregated[i, start_aggregated] = aggregated_shoots

                aggregated[i, start_aggregated + 1] = aggregated_browsed
                
                aggregated[i, start_aggregated + 2] = data[i + 2, start_unaggregated+8]

    return aggregated

In [23]:
# Testing aggregate_data.
test_aggregated = aggregate_data(test_data)
np.savetxt('./Tests/test_aggregate_data.csv', test_aggregated, fmt='%s', delimiter=',')

In [4]:
def combine_forest(paths:List[np.ndarray]) -> np.ndarray:
    '''Combines data from different sampling paths within a forest into a single numpy array.
    This function assumes that the sampling paths have the same species recorded in the same order.
    
    :param: paths: a list containing numpy arrays with the data from the different paths.
    :return: a numpy array with the combined forest data.'''

    combined = paths[0]

    if len(paths) > 1:
        for array in paths[1:]:
            combined = np.concatenate((combined, array[1:,:]))

    return combined

In [25]:
# Testing combine_forest.
import math

test_paths = []
test_directory = 'Forests/Nyugat_Mátra/'

for file_name in os.listdir(test_directory):
    if file_name.endswith('xlsx'):
            read_data = read_forest_data(test_directory + file_name)

            test_paths.append(aggregate_data(read_data))

test_combined = combine_forest(test_paths)

np.savetxt('./Tests/test_combine_forest.csv', test_combined, fmt='%s', delimiter=',')

print('Shapes of the separate arrays:')
for path in test_paths:
      print(np.shape(path))
print('Shape of the combined array:', np.shape(test_combined))

/var/folders/t3/fn5x2gps4wq728lxf9qrv_lm0000gn/T/ipykernel_40721/2886446634.py:14: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  df = pd.read_excel(file_name, sheet_name=SHEET_NAME, skiprows=3, usecols=f'{START_COLUMN}:ZZZ', header=None, engine='openpyxl').reset_index(drop=True)
/var/folders/t3/fn5x2gps4wq728lxf9qrv_lm0000gn/T/ipykernel_40721/2886446634.py:14: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  df = pd.read_excel(file_name, sheet_name=SHEET_NAME, skiprows=3, usecols=f'{START_COLUMN}:ZZZ', header=None, engine='openpyxl').reset_index(drop=True)


Shapes of the separate arrays:
(101, 69)
(101, 69)
(101, 69)
Shape of the combined array: (301, 69)


/var/folders/t3/fn5x2gps4wq728lxf9qrv_lm0000gn/T/ipykernel_40721/2886446634.py:14: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  df = pd.read_excel(file_name, sheet_name=SHEET_NAME, skiprows=3, usecols=f'{START_COLUMN}:ZZZ', header=None, engine='openpyxl').reset_index(drop=True)


In [5]:
def check_data(data: np.ndarray, file_name: str, property_count: int) -> None:
    '''Checks data for recording errors.
    :param: data: numpy array containing recorded data.
    :param: file_name: string containing the file name from which the data was read.
    :param: property_count: integer showing the number of properties read by species.
    '''

    nr_species = int(np.shape(data)[1] / property_count)

    # Checking for smaller all shoot count than freshly browsed shoots.
    # Assumes that all shoots is the first property per species and freshly browsed is the second.
    for i in range(1, len(data)):
        for j in range(nr_species):
            if data[i, j*property_count] < data[i, j*property_count + 1]:
                print(file_name, data[0, j*property_count], i)

## Constructing the database

### Reading files
The following cell reads the data files into a dictionary in which the keys are names of the forests, and the values are numpy arrays containing the height-level aggregated browsing data from each of the 3 sampling paths in the forest.

In [6]:
%%capture

forests: Dict[str, np.ndarray] = {}

root_folder = './Forests/'

log = []

for folder_name in os.listdir(root_folder):
    folder_path = os.path.join(root_folder, folder_name)

    if os.path.isdir(folder_path):
        paths_data = []

        for file_name in os.listdir(folder_path):
            if file_name.endswith('xlsx'):
                    try:
                        log.append('\n' + file_name)
                        
                        path_data = aggregate_data(read_forest_data(folder_path + '/' + file_name))

                        paths_data.append(path_data)

                    except:
                        warnings.warn(f'Problem with loading {file_name}.', UserWarning)

        combined = combine_forest(paths_data)

        forests[folder_name] = combined

In [7]:
with open('log.txt', 'w') as file:
    for item in log:
        file.write(item + '\n')

In [8]:
# Saving dictionary duplicate so that the reading does not need to be done again.
forests_saved = forests.copy()

# Printing summary of read data:

print('Forest regions:')
print(list(forests.keys()), end='\n\n')

print('Total sampling points:')
print(sum([np.shape(data)[0] - 1 for data in forests.values()]))

Forest regions:
['Gyöngyöstarján Világos-hegy', 'Mátrabérc-Fallóskút', 'Recsk', 'Gyöngyöspata Havas', 'Gyöngyös Sár-hegy', 'Nyugat_Mátra', 'Mátra-Észak']

Total sampling points:
2100


### Calculating species distribution properties within forests
Species coverage and % of all shoots

In [9]:
# For each forest, a table is created, where for each species
# the species coverage (% sampling points) and the species
# % contribution to all shoots in the forest is stored.

import math

distributions: Dict[str, np.ndarray] = {}

for forest, data in forests.items():
    sampling_points = len(data) - 1
    all_shoots = 0

    unique_species, indices = np.unique([' '.join(cell.split(' ')[:-2]) for cell in data[0, :]], return_index=True)
    species = unique_species[np.argsort(indices)]
    nr_species = len(species)

    distributions[forest] = np.concatenate((species[np.newaxis, :].astype(object), np.zeros((2, nr_species))))

    for i in range(1,len(data)):
        for j in range(nr_species):
            offered_shoots = data[i, j*3]
            
            if offered_shoots != 0:
                distributions[forest][1,j] += 1
                distributions[forest][2,j] += offered_shoots

                all_shoots += offered_shoots
    
    #Normalizing species data with nr of sampling points / shoots in the forest.
    coverages = distributions[forest][1, :] / sampling_points
    supplyPerces = distributions[forest][2, :] / all_shoots

    distributions[forest][1, :] = coverages
    distributions[forest][2, :] = supplyPerces

### Adding distribution properties to dataset

In [10]:
# The coverage and % of all shoots for each species is inserted at each sampling point in each forest.

for forest, data in forests.items():

    coverage_values = distributions[forest][1, :][np.newaxis, :]
    supplyPerc_values = distributions[forest][2, :][np.newaxis, :]
    
    species = distributions[forest][0, :][np.newaxis, :]
    nr_species = np.shape(species)[1]

    updated_data = data

    for i in range(nr_species):
        index_coverage = i * 3 + 3 + i*2
        index_supplyPerc = i * 3 + 4 + i*2

        #species_coverage = np.array([species[0, i] + ' forest coverage'])[np.newaxis, 1]
        #species_supplyPerc = np.array([species[0, i] + ' supply percentage'])[np.newaxis, 1]

        coverage_value = coverage_values[0, i]
        coverage_array = np.tile(coverage_value, (np.shape(updated_data)[0], 1)).astype(object)
        coverage_array[0,0] = species[0, i] + ' forest coverage'

        supplyPerc_value = supplyPerc_values[0, i]
        supplyPerc_array = np.tile(supplyPerc_value, (np.shape(updated_data)[0], 1)).astype(object)
        supplyPerc_array[0,0] = species[0, i] + ' supply percentage'

        updated_data = np.insert(updated_data, [index_coverage], coverage_array, axis=1)
        updated_data = np.insert(updated_data, [index_supplyPerc], supplyPerc_array, axis=1)

    forests[forest] = updated_data

In [33]:
# Test distribution properties.
np.savetxt('./Tests/test_distribution.csv', forests['Gyöngyöstarján Világos-hegy'], fmt='%s', delimiter=',')

`NOTE:` The cells below assume that there are 5 properties stored for each species per sampling point (nr of offered shoots, nr of freshly browsed shoots, nr of old browsed shoots, coverage in the area, supply percentage). If in the future properties are added/deleted, the cells below need to be modified accordingly.

### Aggregating unknown species

Data on different unknown species is aggregated into a single species labeled 'Unknown'. If there are no unknown species in the forest, the columns for 'Unknown' are filled with zero values.

In [11]:
for forest, data in forests.items():
    nr_species = int(np.shape(data)[1] / 5)
    rows = len(data)
    
    #print(f'Original: {np.shape(data)}')

    unknown_cols = []

    unknown_allShoots = np.zeros((rows, 1)).astype(object)
    unknown_allShoots[0, 0] = 'Unknown all shoots'

    unknown_freshBrowsed = np.zeros((rows, 1)).astype(object)
    unknown_freshBrowsed[0, 0] = 'Unknown freshly browsed'

    unknown_oldBrowsed = np.zeros((rows, 1)).astype(object)
    unknown_oldBrowsed[0, 0] = 'Unknown old browsed'

    unknown_coverage = np.zeros((rows, 1)).astype(object)
    unknown_coverage[0, 0] = 'Unknown forest coverage'

    unknown_supplyPerc = np.zeros((rows, 1)).astype(object)
    unknown_supplyPerc[0, 0] = 'Unknown supply percentage'

    for i in range(nr_species):
        species = data[0, i*5].lower()

        if species.startswith('faj') or 'ismeretlen' in species:
            unknown_cols.extend([i*5, i*5+1, i*5+2, i*5+3, i*5+4])

    #print(f'Unknown: {unknown_cols}')

    for i in range(1, len(data)):

        allShoots_columns = [col for col in unknown_cols if col % 5 == 0]
        unknown_allShoots[i, 0] = np.sum(data[i, [allShoots_columns]])

        freshBrowsed_columns = [col for col in unknown_cols if col % 5 == 1]
        unknown_freshBrowsed[i, 0] = np.sum(data[i, [freshBrowsed_columns]])

        oldBrowsed_columns = [col for col in unknown_cols if col % 5 == 2]
        unknown_oldBrowsed[i, 0] = np.sum(data[i, [oldBrowsed_columns]])

        coverage_columns = [col for col in unknown_cols if col % 5 == 3]
        unknown_coverage[i, 0] = np.sum(data[i, [coverage_columns]])

        supplyPerc_columns = [col for col in unknown_cols if col % 5 == 4]
        unknown_supplyPerc[i, 0] = np.sum(data[i, [supplyPerc_columns]])

    unknown_array = np.concatenate((unknown_allShoots, unknown_freshBrowsed, unknown_oldBrowsed, \
                                    unknown_coverage, unknown_supplyPerc), axis=1)
    
    data = np.delete(data, [unknown_cols], axis=1)
    #print(f'Deleted: {np.shape(data)}')
    data = np.insert(data, [0], unknown_array, axis=1)
    #print(f'Inserted: {np.shape(data)}')

    forests[forest] = data

### Saving species per forest.

In [12]:
species_per_forest = dict()

for forest, data in forests.items():
    forest_species = []

    nr_species = int(np.shape(data)[1] / 5)

    for i in range(nr_species):
        sp = ' '.join(data[0, i*5].split(' ')[:-2])

        forest_species.append(sp)

    species_per_forest[forest] = forest_species

print(species_per_forest)

{'Gyöngyöstarján Világos-hegy': ['Unknown', 'Kocsánytalan tölgy', 'Kocsányos tölgy', 'Csertölgy', 'Magas kőris', 'Virágos kőris', 'Gyertyán', 'Bükk', 'Hegyi juhar', 'Korai juhar', 'Mezei juhar', 'Erdei fenyő', 'Akác', 'Fagyal', 'Galagonya', 'Húsos som', 'Veresgyűrűs som', 'Kökény', 'Szeder', 'Vadrózsa', 'Bodza', 'Tatárjuhar'], 'Mátrabérc-Fallóskút': ['Unknown', 'Kocsánytalan tölgy', 'Kocsányos tölgy', 'Csertölgy', 'Magas kőris', 'Virágos kőris', 'Gyertyán', 'Bükk', 'Hegyi juhar', 'Korai juhar', 'Mezei juhar', 'Erdei fenyő', 'Akác', 'Fagyal', 'Galagonya', 'Húsos som', 'Veresgyűrűs som', 'Kökény', 'Szeder', 'Vadrózsa', 'Bodza', 'Madárcseresznye', 'Mogyoró'], 'Recsk': ['Unknown', 'Kocsánytalan tölgy', 'Kocsányos tölgy', 'Csertölgy', 'Magas kőris', 'Virágos kőris', 'Gyertyán', 'Bükk', 'Hegyi juhar', 'Korai juhar', 'Mezei juhar', 'Erdei fenyő', 'Akác', 'Fagyal', 'Galagonya', 'Húsos som', 'Veresgyűrűs som', 'Kökény', 'Szeder', 'Vadrózsa', 'Bodza', 'Mezei szil', 'Berkenye'], 'Gyöngyo

In [13]:
import csv
from itertools import zip_longest

species_columns = zip_longest(*species_per_forest.values(), fillvalue='')

with open('species_per_forest.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(species_per_forest.keys())

    writer.writerows(species_columns)

### Combining forest data

The data on different forest regions are combined into a single table consisting of sampling points as instances (rows). Columns for species which are not found in a given forest are filled with 0 values.

#### Determining total number of species in all regions

In [14]:
species = np.array([])

for forest, data in forests.items():
    nr_species = int(np.shape(data)[1] / 5)

    species_cols = [i * 5 for i in range(nr_species)]

    for col in species_cols:
        sp = ' '.join(data[0, col].split(' ')[:-2])

        species = np.concatenate((species, np.array([sp])))

unique_species, indices = np.unique(species, return_index=True)
species = unique_species[np.argsort(indices)]
nr_species = len(species)

print(f'Number of unique species in the dataset: {nr_species}')
print(f'Unique species in the dataset: {species}')


Number of unique species in the dataset: 29
Unique species in the dataset: ['Unknown' 'Kocsánytalan tölgy' 'Kocsányos tölgy' 'Csertölgy'
 'Magas kőris' 'Virágos kőris' 'Gyertyán' 'Bükk' 'Hegyi juhar'
 'Korai juhar' 'Mezei juhar' 'Erdei fenyő' 'Akác' 'Fagyal' 'Galagonya'
 'Húsos som' 'Veresgyűrűs som' 'Kökény' 'Szeder' 'Vadrózsa' 'Bodza'
 'Tatárjuhar' 'Madárcseresznye' 'Mogyoró' 'Mezei szil' 'Berkenye'
 'Vadkörte' 'Bibircses kecskerágó' 'Molyhos tölgy']


In [15]:
nr_rows = 0

for data in forests.values():
    nr_rows += np.shape(data)[0] - 1

print(f'Total number of sampling points in dataset: {nr_rows}')

Total number of sampling points in dataset: 2100


#### Creating combined dataset

In [16]:
# Initializing dataset with headers and all 0 values.
dataset = np.zeros((nr_rows + 1, nr_species * 5)).astype(object)

for i in range(nr_species):
    sp = species[i]

    dataset[0, i*5] = f'{sp} all shoots'
    dataset[0, i*5 + 1] = f'{sp} freshly browsed'
    dataset[0, i*5 + 2] = f'{sp} old browsed'
    dataset[0, i*5 + 3] = f'{sp} forest coverage'
    dataset[0, i*5 + 4] = f'{sp} supply percentage'

In [17]:
dataset_row = 1

for forest, data in forests.items():
    rows, cols = np.shape(data)

    for i in range(1, rows):
        for j in range(cols):
            header = data[0, j]

            dataset_col = np.where(dataset[0] == header)[0][0]

            dataset[dataset_row, dataset_col] = data[i, j]

        dataset_row += 1


### Additional processing.
The cells below contain additional processing steps on the combined dataset.

#### Normalizing freshly browsed shoots.
To form ratios, the number of freshly browsed shoots is divided by the number of total shoots at the sampling point.

In [18]:
rows, cols = np.shape(dataset)

for j in range(cols):
    if 'freshly browsed' in dataset[0, j]:
        for i in range(1, rows):
            all_offered = dataset[i, j-1]
            
            if all_offered != 0:
                dataset[i, j] = dataset[i, j] / all_offered

            elif dataset[i,j] != 0:     # Correcting potential recording mistakes.
                dataset[i, j] = 0

#### Removing old browsed shoots.
Old browsed shoots are removed from the dataset as they will not be used in the machine learning analyis. This is an optional step, if this data is needed as well, simply skip the cell below.

In [19]:
headers = dataset[0, :].tolist()

old_browsed_cols = [index for index, header in enumerate(headers) if 'old browsed' in header]

dataset = np.delete(dataset, [old_browsed_cols], axis=1)

#### Changing Hungarian to Latin names

In [20]:
names_dict = {'Kocsánytalan tölgy' : 'Quercus petraea', 'Kocsányos tölgy' : 'Quercus robur', 'Csertölgy' : 'Quercus cerris',  'Magas kőris' : 'Fraxinus excelsior', \
    'Virágos kőris' : 'Fraxinus ornus', 'Gyertyán' : 'Carpinus betulus', 'Bükk' : 'Fagus sylvatica', 'Hegyi juhar' : 'Acer pseudoplatanus', 'Korai juhar' : 'Acer platanoides', \
        'Mezei juhar' : 'Acer campestre', 'Erdei fenyő' : 'Pinus sylvestris', 'Akác' : 'Robinia pseudoacacia', 'Fagyal' : 'Ligustrum vulgare ', 'Galagonya' : 'Crataegus monogyna', \
            'Húsos som' : 'Cornus mas', 'Veresgyűrűs som' : 'Cornus sanguinea', 'Kökény' : 'Prunus spinosa', 'Szeder' : 'Rubus fruticosus ', \
                'Vadrózsa' : 'Rosa canina ', 'Bodza' : 'Sambucus nigra', 'Tatárjuhar' : 'Acer tataricum', 'Madárcseresznye': 'Prunus avium ', 'Mogyoró' : 'Corylus avellana', \
                    'Mezei szil' : 'Ulmus minor ', 'Berkenye' : 'Sorbus aucuparia', 'Vadkörte' : 'Pyrus pyraster ', 'Bibircses kecskerágó' : 'Euonymus verrucosus', \
                        'Molyhos tölgy' : 'Quercus pubescens'}

In [25]:
for j in range(np.shape(dataset)[1]):
    species = ' '.join(dataset[0, j].split(' ')[:-2])
    feature = ' '.join(dataset[0, j].split(' ')[-2:])

    if species != 'Unknown':
        dataset[0, j] = f'{names_dict[species]} {feature}'

#### Save dataset as csv.

We mark it with 'raw' to allow for additional processing after data inspection.

In [26]:
np.savetxt('./Data/dataset_raw.csv', dataset, fmt='%s', delimiter=',')

### Saving predictor and target values.

In [27]:
target_columns = [i for i in range(np.shape(dataset)[1]) if 'freshly browsed' in dataset[0, i]]

target_values = dataset[:, target_columns]

print(f'Shape of target value array: {np.shape(target_values)}')

Shape of target value array: (2101, 29)


In [28]:
np.savetxt('./Data/freshly_browsed_raw.csv', target_values, fmt='%s', delimiter=',')

#### Saving target values with distinction between 0% browsed and 0 supply.

In [29]:
target_values_distinguished = target_values.copy()

noSupply_value = -0.00001         # The value to use if there was no shoot supply.

rows, cols = np.shape(target_values_distinguished)

for i in range(rows):
    for j in range(cols):
        if dataset[i, j*4] == 0:        # If there was no supply from that species.
            target_values_distinguished[i, j] = noSupply_value

In [30]:
np.savetxt('./Data/freshly_browsed_dist_raw.csv', target_values_distinguished, fmt='%s', delimiter=',')

In [31]:
predictor_columns = [i for i in range(np.shape(dataset)[1]) if 'freshly browsed' not in dataset[0, i]]

predictor_values = dataset[:, predictor_columns]

print(f'Shape of predictor values array: {np.shape(predictor_values)}')

Shape of predictor values array: (2101, 87)


In [32]:
np.savetxt('./Data/predictors_raw.csv', predictor_values, fmt='%s', delimiter=',')